In [12]:
import pandas as pd

df_dict = pd.read_csv(r'C:\Users\Aluno\Projeto\final-project\data\P18_dicionario_dados.csv', encoding='utf-8', sep=',')
df_cancel = pd.read_csv(r'C:\Users\Aluno\Projeto\final-project\data\P18_cancelamentos.csv', encoding='utf-8', sep=',')
df_contratos = pd.read_csv(r'C:\Users\Aluno\Projeto\final-project\data\P18_contratos.csv', encoding='utf-8', sep=',')
df_clientes= pd.read_csv(r'C:\Users\Aluno\Projeto\final-project\data\P18_clientes.csv', encoding='utf-8', sep=',')

1. Quantos planos de seguro temos?

In [13]:
df_contratos['tipo_seguro'].nunique()
df_contratos['tipo_seguro'].value_counts()

tipo_seguro
Auto           2429
Empresarial    2423
Vida           2406
Saúde          2390
Residencial    2352
Name: count, dtype: int64

2. Quais planos possuem mais cancelamento?

In [15]:
# Unir Contratos com Cancelamentos
df_cancelados = df_contratos.merge(df_cancel, on='id_contrato', how='inner')

# Contar Cancelamentos por Tipo de Seguro
df_cancelados['tipo_seguro'].value_counts()

tipo_seguro
Saúde          528
Empresarial    522
Residencial    512
Vida           489
Auto           469
Name: count, dtype: int64

3. Qual o tempo de contrato médio dos clientes?

In [16]:
# Converter Datas
df_contratos['data_inicio'] = pd.to_datetime(df_contratos['data_inicio'])
df_contratos['data_fim'] = pd.to_datetime(df_contratos['data_fim'])

# Criar Coluna com Duração em Dias
df_contratos['duracao_contrato'] = (df_contratos['data_fim'] - df_contratos['data_inicio']).dt.days

# Média Geral
df_contratos['duracao_contrato'].mean()

424.2079166666667

4. Qual o tempo médio dos contratos cancelados?

In [18]:
df_cancelados['data_inicio'] = pd.to_datetime(df_cancelados['data_inicio'])
df_cancelados['data_fim'] = pd.to_datetime(df_cancelados['data_fim'])

df_cancelados['duracao_contrato'] = (df_cancelados['data_fim'] - df_cancelados['data_inicio']).dt.days
df_cancelados['duracao_contrato'].mean()

415.67857142857144

9. Houve troca de plano antes do cancelamento?

In [19]:
# Ver Clientes com Mais de um Tipo de Seguro
df_contratos.sort_values(by=['id_cliente', 'data_inicio'], inplace=True)
df_troca = df_contratos.groupby('id_cliente')['tipo_seguro'].nunique()

# Clientes que Já Tiveram Mais de um Tipo
trocaram = df_troca[df_troca > 1].index

# Quantos Desses Também Cancelaram
cancelaram_e_trocaram = df_cancelados[df_cancelados['id_cliente'].isin(trocaram)]
print("Cancelaram após trocar de plano:", len(cancelaram_e_trocaram))

Cancelaram após trocar de plano: 2184


10. Clientes que renovaram contrato ao menos uma vez cancelam menos?

In [20]:
# Clientes que Renovaram
renovaram = df_contratos[df_contratos['renovado_automaticamente'] == True]
nao_renovaram = df_contratos[df_contratos['renovado_automaticamente'] == False]

# Com Cancelamento
renovaram_cancelaram = renovaram[renovaram['id_contrato'].isin(df_cancel['id_contrato'])]
nao_renovaram_cancelaram = nao_renovaram[nao_renovaram['id_contrato'].isin(df_cancel['id_contrato'])]

# Taxas
print("Taxa de cancelamento com renovação:", len(renovaram_cancelaram) / len(renovaram))
print("Taxa sem renovação:", len(nao_renovaram_cancelaram) / len(nao_renovaram))

Taxa de cancelamento com renovação: 0.20939464108501488
Taxa sem renovação: 0.2106147127981189


11. Canal de venda influencia na fidelidade?

In [22]:
df_contratos['cancelado'] = df_contratos['id_contrato'].isin(df_cancel['id_contrato'])

df_contratos.groupby('canal_venda')['cancelado'].mean().sort_values(ascending=False)

canal_venda
Agência Física    0.211807
App               0.211277
Corretor          0.209721
Site              0.207192
Name: cancelado, dtype: float64